In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys
import numpy as np
import pandas as pd
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict

# packages for Survival analyis
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.metrics import concordance_index_censored, cumulative_dynamic_auc
from sksurv.ensemble import RandomSurvivalForest, ExtraSurvivalTrees
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.svm import FastKernelSurvivalSVM
from sksurv.nonparametric import kaplan_meier_estimator



## -----------------------------------------<br><br>Import data

In [ ]:
# import train and test set
# external validation is imported to compute KM curves
X_train = pd.read_csv('data/survival/OS/X_train.csv', index_col=0)
X_test = pd.read_csv('data/survival/OS/X_val.csv', index_col=0)
X_ext = pd.read_csv('data/survival/OS/X_test.csv', index_col=0)
y_train = pd.read_csv('data/survival/OS/y_train.csv', index_col=0)
y_test = pd.read_csv('data/survival/OS/y_val.csv', index_col=0)
y_ext = pd.read_csv('data/survival/OS/y_test.csv', index_col=0)

y_train=y_train.squeeze()
y_test=y_test.squeeze()
y_ext=y_ext.squeeze()

from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10)

In [ ]:
# the event variable must be boolean
y_train['STATUS OS']=y_train['STATUS OS'].astype(bool)
y_test['STATUS OS']=y_test['STATUS OS'].astype(bool)
y_ext['STATUS OS']=y_ext['STATUS OS'].astype(bool)
# targets must be an array (not a dataframe)
y_tr=y_train.to_records(index=False)
y_t=y_test.to_records(index=False)
y_e=y_ext.to_records(index=False)

### KAPLAN-MEIER ESTIMATION

In [ ]:
# KAPLAN MEIER ESTIMATION FOR SURVIVAL CURVE 
y =np.concatenate([y_tr,y_t,y_e])
X =pd.concat([X_train, X_test,X_ext],axis=0)

t_km, surv_km, conf_int = kaplan_meier_estimator(y['STATUS OS'],y['OS'], conf_type='log-log')

plt.step(t_km,surv_km, where="post")
plt.fill_between(t_km, conf_int[0], conf_int[1], alpha=0.25, step="post")
plt.ylim(0, 1)
plt.grid()
plt.ylabel("Survival probability")
plt.xlabel("Months from IO baseline")

path_save = 'results/survival/OS/K-M_with_confidence.png'
#plt.savefig(path_save, format='png')

### Confidence interval with Bootstrapping

In [ ]:
pd.DataFrame(y['OS']).median()

In [ ]:
n_iter = 1000
import random, math
results = []

for i in range(n_iter):
    x = [random.randint(0,X.shape[0]-1) for i in range(0,math.floor(X.shape[0]/2))]
    y_bs = y.copy()
    y_bs = y_bs[x]
    res = pd.DataFrame(y_bs['OS']).median()
    results.append(res)

results = np.array(results).squeeze()
results.sort()
# confidence intervals
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(results, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(100, np.percentile(results, p))
print('%.0f%% confidence interval %.3f and %.3f' % (alpha*100, lower, upper))

## ------------------------------- <br><br>FEATURE SELECTION - ELASTIC NET

In [ ]:
# FUNCTION FOR PLOTTING TREND OF REGRESSION COEFFICIENTS

def plot_coefficients(coefs, n_highlight):
    _, ax = plt.subplots(figsize=(15, 15))
    n_features = coefs.shape[0]
    alphas = coefs.columns
    for row in coefs.itertuples():
        ax.semilogx(alphas, row[1:], ".-", label=row.Index)

    alpha_min = alphas.min()
    top_coefs = coefs.loc[:, alpha_min].map(abs).sort_values().tail(n_highlight)
    for name in top_coefs.index:
        coef = coefs.loc[name, alpha_min]
        plt.text(
            alpha_min, coef, name + "   ",
            horizontalalignment="right",
            verticalalignment="center",
            fontsize=20
        )

    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax.grid(True)
    ax.set_xlabel("alpha", fontsize=20)
    ax.set_ylabel("coefficient", fontsize=20)
    ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 20)
    ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 20)
    
    # put the value of alpha selected to plot the vertical blue line inside the trend of regresison coefficients
    #ax.axvline(0.030816829260251342, c="b")

    
    path='results\survival\OS\EN_coefs_trend.png'
    #plt.savefig(path, format='png', bbox_inches = 'tight')

In [ ]:
# fit EN model
cox_elastic_net = CoxnetSurvivalAnalysis(l1_ratio=0.9, alpha_min_ratio=0.01)
cox_elastic_net.fit(X_train, y_tr)

In [ ]:
# extract the values of alpha found by EN and use grid search to select its optimal value (maximization of C-index)
estimated_alphas=cox_elastic_net.alphas_
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

gcv = GridSearchCV(
    CoxnetSurvivalAnalysis(l1_ratio=0.9, alpha_min_ratio=0.01),
    param_grid={"alphas": [[v] for v in estimated_alphas]},
    cv=cv,
    error_score=0.5,
    n_jobs=1)

gcv=gcv.fit(X_train,y_tr)


cv_results = pd.DataFrame(gcv.cv_results_)

In [ ]:
# PLOT TREND OF C-INDEX IN FUNCTION OF ALPHA
alphas = cv_results.param_alphas.map(lambda x: x[0])
mean = cv_results.mean_test_score
std = cv_results.std_test_score

fig, ax = plt.subplots(figsize=(9, 6))
ax.plot(alphas, mean)
ax.fill_between(alphas, mean - std, mean + std, alpha=.15)
ax.set_xscale("log")
ax.set_ylabel("concordance index")
ax.set_xlabel("alpha")
ax.axvline(gcv.best_params_["alphas"][0], c="C1")
ax.axhline(0.5, color="grey", linestyle="--")
ax.grid(True)
path='results\survival\OS\EN_C_IND_trend.png'
#plt.savefig(path, format='png',bbox_inches = 'tight')

In [ ]:
# PLOT COEFFICIENTS FOR THE OPTIMAL ALPHA

best_model = gcv.best_estimator_
best_coefs = pd.DataFrame(
    best_model.coef_,
    index=X_train.columns,
    columns=["coefficient"]
)

non_zero = np.sum(best_coefs.iloc[:, 0] != 0)
print("Number of non-zero coefficients: {}".format(non_zero))

non_zero_coefs = best_coefs.query("coefficient != 0")
coef_order = non_zero_coefs.abs().sort_values("coefficient").index

_, ax = plt.subplots(figsize=(6, 8))
non_zero_coefs.loc[coef_order].plot.barh(ax=ax, legend=False)
ax.set_xlabel("coefficient")
ax.grid(True)

path='results\survival\OS\EN_sel_coefs.png'
#plt.savefig(path, format='png', bbox_inches = 'tight')

In [ ]:
# BEST ALPHA SELECTED
gcv.best_params_["alphas"][0]

In [ ]:
# PLOT COEFFICIENTS TREND WITH THE OPTIMAL ALPHA HIGHLIGHTED

coefficients_elastic_net = pd.DataFrame(
    cox_elastic_net.coef_,
    index=X_train.columns,
    columns=np.round(cox_elastic_net.alphas_, 8)
)

plot_coefficients(coefficients_elastic_net,5)

In [ ]:
# SELECTED FEATURES
coefs_best=best_coefs.index[np.where(best_coefs['coefficient']!=0)[0]]

In [ ]:
# COMMENT THE COUPLE OF LINES IF YOU WANT TO TRAIN WITH EN OR WITH ALL THE FEATURES  

#X_tr = X_train.copy()
#X_t = X_test.copy()

X_tr=X_train.loc[:,coefs_best]
X_t=X_test.loc[:,coefs_best]

## ---------------------------------<br><br> SURVIVAL ANALYSIS

In each section a model is trained, with the relative hyperparameters tuned through gridsearch;
the c-index is calculated

### <br><br> CPH

In [ ]:
cox=CoxPHSurvivalAnalysis()
param_grid = {'n_iter': [5,10,15,20,50,100]}
cv=StratifiedKFold(n_splits = 10, shuffle = True)
gcv = GridSearchCV(cox, param_grid, return_train_score=True, cv=cv)
gcv.fit(X_tr, y_tr)

print("Best score: {f}".format(f=gcv.best_score_))
print("Parameters: {f}".format(f=gcv.best_params_))

In [ ]:
ci_cox_cv = gcv.best_score_ 
best_cox = gcv.best_estimator_

ci_cox_train = concordance_index_censored(y_tr["STATUS OS"], y_tr["OS"], best_cox.predict(X_tr))
ci_cox_test = concordance_index_censored(y_t["STATUS OS"], y_t["OS"], best_cox.predict(X_t))

print("COX PROPORTIONAL HAZARDS\n----------------------")
print("C-index TRAIN: {f}".format(f=ci_cox_train[0]))
print("C-index CV: {f}".format(f=ci_cox_cv))
print("C-index TEST: {f}".format(f=ci_cox_test[0]))

### <br><br> RSF

In [ ]:
rsf = RandomSurvivalForest()
param_grid = {
              'max_depth':[2,3],
              'min_samples_leaf':[2,3,4],
              'min_samples_split': [2,3,4],
              'n_estimators': [50,100]
             }

gcv1 = GridSearchCV(rsf, param_grid, return_train_score=True, cv=cv, verbose=1)
gcv1.fit(X_tr, y_tr)
print("Best score: {f}".format(f=gcv1.best_score_))
print("Parameters: {f}".format(f=gcv1.best_params_))

In [ ]:
ci_rsf_cv = gcv1.best_score_ 
best_rsf = gcv1.best_estimator_

ci_rsf_train = concordance_index_censored(y_tr["STATUS OS"], y_tr["OS"], best_rsf.predict(X_tr))
ci_rsf_test = concordance_index_censored(y_t["STATUS OS"], y_t["OS"], best_rsf.predict(X_t))

print("RANDOM SURVIVAL FOREST\n----------------------")
print("C-index TRAIN: {f}".format(f=ci_rsf_train[0]))
print("C-index CV: {f}".format(f=ci_rsf_cv))
print("C-index TEST: {f}".format(f=ci_rsf_test[0]))

### <br><br> GB

In [ ]:
gb = GradientBoostingSurvivalAnalysis()

param_grid={
           'learning_rate':[0.001,0.01,0.1],
           'min_samples_leaf':[3,4,6],
            'min_samples_split': [3,4,6],
           'max_depth':[2],
           'n_estimators': [30,50,90]
           }


gcv2 = GridSearchCV(gb, param_grid, return_train_score=True, cv=cv, verbose=30, n_jobs=-1)
gcv2.fit(X_tr, y_tr)
print("Best score: {f}".format(f=gcv2.best_score_))
print("Parameters: {f}".format(f=gcv2.best_params_))

In [ ]:
ci_gb_cv = gcv2.best_score_ 
best_gb = gcv2.best_estimator_

ci_gb_train = concordance_index_censored(y_tr["STATUS OS"], y_tr["OS"], best_gb.predict(X_tr))
ci_gb_test = concordance_index_censored(y_t["STATUS OS"], y_t["OS"], best_gb.predict(X_t))

print("GRADIENT BOOSTING\n----------------------")
print("C-index TRAIN: {f}".format(f=ci_gb_train[0]))
print("C-index CV: {f}".format(f=ci_gb_cv))
print("C-index TEST: {f}".format(f=ci_gb_test[0]))

### <br><br>SSVM

In [ ]:
# you can't have 0 values in the training SVM, so you make them very close to it
y1 = y_train.copy()
y1['OS']=y1['OS'].astype('float64')
y1=y1.to_records(index=False)

y1[np.where(y1['OS']==0)[0]] = 1e-8

In [ ]:
fks=FastKernelSurvivalSVM()
param_grid={"alpha":[0.01,0.1,1,10], "kernel":['linear','rbf'], "gamma":[0.01,0.1,1,10]}

gcv3 = GridSearchCV(fks, param_grid, return_train_score=True, cv=cv, verbose=1, n_jobs=-1)
gcv3.fit(X_tr, y1)
print("Best score: {f}".format(f=gcv3.best_score_))
print("Parameters: {f}".format(f=gcv3.best_params_))

In [ ]:
ci_fks_cv = gcv3.best_score_ 
best_fks = gcv3.best_estimator_

ci_fks_train = concordance_index_censored(y_tr["STATUS OS"], y_tr["OS"], best_fks.predict(X_tr))
ci_fks_test = concordance_index_censored(y_t["STATUS OS"], y_t["OS"], best_fks.predict(X_t))

print("FAST SSVM\n----------------------")
print("C-index TRAIN: {f}".format(f=ci_fks_train[0]))
print("C-index CV: {f}".format(f=ci_fks_cv))
print("C-index TEST: {f}".format(f=ci_fks_test[0]))


### <br><br> EST

In [ ]:
et = ExtraSurvivalTrees()
param_grid = {
              'max_depth':[2,3],
              'min_samples_leaf':[2,3,4,5],
              'min_samples_split': [2,3,4,5],
              'n_estimators': [50,100]
             }

gcv4 = GridSearchCV(et, param_grid, return_train_score=True, cv=cv, verbose=1)
gcv4.fit(X_tr, y_tr)
print("Best score: {f}".format(f=gcv4.best_score_))
print("Parameters: {f}".format(f=gcv4.best_params_))

In [ ]:
ci_et_cv = gcv4.best_score_ 
best_est = gcv4.best_estimator_

ci_et_train = concordance_index_censored(y_tr["STATUS OS"], y_tr["OS"], best_est.predict(X_tr))
ci_et_test = concordance_index_censored(y_t["STATUS OS"], y_t["OS"], best_est.predict(X_t))

print("EXTRA SURVIVAL TREES\n----------------------")
print("C-index TRAIN: {f}".format(f=ci_et_train[0]))
print("C-index CV: {f}".format(f=ci_et_cv))
print("C-index TEST: {f}".format(f=ci_et_test[0]))


## Save Models

In [ ]:
names = ['CPH', 'RSF', 'GB', 'SSVM', 'EST']

In [ ]:
import pickle
# TO SAVE MODELS AFTER EN SELECTION
path = 'results/survival/OS/MODELS/{n}.pkl'.format(n=names[0])
pickle.dump(best_cox, open(path, 'wb'))
path = 'results/survival/OS/MODELS/{n}.pkl'.format(n=names[1])
pickle.dump(best_rsf, open(path, 'wb'))
path = 'results/survival/OS/MODELS/{n}.pkl'.format(n=names[2])
pickle.dump(best_gb, open(path, 'wb'))
path = 'results/survival/OS/MODELS/{n}.pkl'.format(n=names[3])
pickle.dump(best_fks, open(path, 'wb'))
path = 'results/survival/OS/MODELS/{n}.pkl'.format(n=names[4])
pickle.dump(best_est, open(path, 'wb'))

In [ ]:
# TO SAVE MODELS WITH ALL THE FEATURES
path = 'results/survival/OS/MODELS(ALL)/{n}.pkl'.format(n=names[0])
pickle.dump(best_cox, open(path, 'wb'))
path = 'results/survival/OS/MODELS(ALL)/{n}.pkl'.format(n=names[1])
pickle.dump(best_rsf, open(path, 'wb'))
path = 'results/survival/OS/MODELS(ALL)/{n}.pkl'.format(n=names[2])
pickle.dump(best_gb, open(path, 'wb'))
path = 'results/survival/OS/MODELS(ALL)/{n}.pkl'.format(n=names[3])
pickle.dump(best_fks, open(path, 'wb'))
path = 'results/survival/OS/MODELS(ALL)/{n}.pkl'.format(n=names[4])
pickle.dump(best_est, open(path, 'wb'))

## Dynamic AUC

In [ ]:
X_tra = X_tr.copy()
X_te =X_t.copy()


In [ ]:
trained_models = []
survs = []


for j in range(0, len(names)):

    path = 'results/survival/OS/MODELS/{n}.pkl'.format(n=names[j])
    # if you want to upload models with all the features
    #path = 'results/survival/OS/MODELS(ALL)/{n}.pkl'.format(n=names[j])  
    
    train_mod=pickle.load(open(path, 'rb'))
    if(j!=3):
        surv= train_mod.predict_survival_function(X_te)
    else:
        surv = [0]

    trained_models.append(train_mod)
    survs.append(surv)

In [ ]:
times = np.linspace(min(y_t['OS']),max(y_t['OS'])-0.001,30)

In [ ]:
# calculate Dynamic AUC

cox_risk_scores=trained_models[0].predict(X_te)
cox_auc, cox_mean_auc = cumulative_dynamic_auc(
    y_tr, y_t, cox_risk_scores, times
)

rsf_risk_scores=trained_models[1].predict(X_te)
rsf_auc, rsf_mean_auc = cumulative_dynamic_auc(
    y_tr, y_t, rsf_risk_scores, times
)


gb_risk_scores=trained_models[2].predict(X_te)
gb_auc, gb_mean_auc = cumulative_dynamic_auc(
    y_tr, y_t, gb_risk_scores, times
)


fks_risk_scores=trained_models[3].predict(X_te)
fks_auc, fks_mean_auc = cumulative_dynamic_auc(
    y_tr, y_t, fks_risk_scores, times
)


et_risk_scores=trained_models[4].predict(X_te)
et_auc, et_mean_auc = cumulative_dynamic_auc(
    y_tr, y_t, et_risk_scores, times
)

print("Average CPH: ",cox_mean_auc)
print("Average RSF: ",rsf_mean_auc)
print("Average GB: ",gb_mean_auc)
print("Average SSVM: ",fks_mean_auc)
print("Average EST: ",et_mean_auc)
print('-------------------------')


### <br><br>  Estimation of Survival Functions

In [ ]:
# KAPLAN MEIER ESTIMATION FOR SURVIVAL CURVE 
y =np.concatenate([y_tr,y_t, y_e])

t_km, surv_km = kaplan_meier_estimator(y['STATUS OS'],y['OS'])

In [ ]:
# put here the best model according to its index inside the vectors, remember that for SSVM you can't estimate the
# survival function!
preds = [[fn(t) for t in trained_models[4].unique_times_] for fn in survs[4]]


In [ ]:
surv=pd.DataFrame(preds)

mean_surv=surv.mean()

t = trained_models[4].unique_times_

# choose only patients that experienced the event

surv['OS']=y_t['OS']
surv['Event']=y_t['STATUS OS']
surv

In [ ]:
# SURVIVAL CURVE FOR PATIENT WITH LOW OS (put the threshold value that you want)
surv_low=surv['OS']<=5
surv_low=surv[surv_low]
surv_low

In [ ]:
# SURVIVAL CURVE FOR PATIENT WITH HIGH OS
surv_high=surv['OS']>=30
surv_high=surv[surv_high]
surv_high

In [ ]:
# PLOT THE SURVIVAL CURVES FOR PATIENTS WITH HIGH AND LOW SIRVIVAL, PUT THE INDEXES OF THE CHOSEN PATIENTS

plt.figure(figsize=(15,5))


plt.subplot(1,2,1)
plt.step(t_km, surv_km, where="post", color="g", label="Kaplan Meier estimation")
plt.step(t, surv.iloc[76,:-2], where="post", color="k", label="Survival curve")
plt.axvline(surv.loc[76,'OS'], linestyle="--", color="b",linewidth=2, label="Time of death")
plt.legend(loc="best", fontsize='small')
plt.title("Patient with OS = {os}".format(os=surv.loc[76,'OS']))
plt.xlabel("Months from baseline of IO")
plt.ylabel("Survival probability")
plt.xlim(left=-1,right=max(y_t['OS']))
plt.ylim(0,1)
plt.grid(True)

plt.subplot(1,2,2)
plt.step(t_km, surv_km, where="post", color="g", label="Kaplan Meier estimation")
plt.step(t, surv.iloc[5,:-2], where="post", color="k", label="Survival curve")
plt.axvline(surv.loc[5,'OS'], linestyle="--", color="b",linewidth=2, label="Time of death")
plt.legend(loc="best", fontsize='small')
plt.title("Patient with OS = {os}".format(os=surv.loc[5,'OS']))
plt.xlabel("Months from baseline of IO")
plt.ylabel("Survival probability")
plt.xlim(left=-1,right=max(y_t['OS']))
plt.ylim(0,1)
plt.grid(True)

path='results\survival\OS\SHAP AND CURVES/Curves_EST_ALL.png'
plt.savefig(path, format='png', bbox_inches = 'tight')
